In [ ]:
import sys
sys.stdout.fileno = lambda : 1

In [ ]:
from testing_fns import setup, load_image
from viz import format_maps
from mmrotate.models import build_loss
import matplotlib.pyplot as plt

In [73]:
model, datasets, cfg, timestamp, meta = setup("configs/fcosr/fcosr_res50_32x4d_fpn_3x_dota10_single.py")

model.bbox_head.cls_loss_fn = build_loss(
    dict(type='QualityFocalLoss', use_sigmoid=True, beta=2.0, reduction='none', loss_weight=1.0))
model.bbox_head.reg_loss_fn = build_loss(
    dict(type='ProbiouLoss', mode='l1', loss_weight=0.0))

2022-05-10 12:26:46,658 - mmrotate - INFO - Environment info:
------------------------------------------------------------
sys.platform: win32
Python: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
CUDA available: True
GPU 0: NVIDIA GeForce RTX 3070 Laptop GPU
CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3
NVCC: Cuda compilation tools, release 11.3, V11.3.58
MSVC: Microsoft (R) C/C++ Optimizing Compiler Version 19.29.30138 for x64
GCC: n/a
PyTorch: 1.11.0
PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 192829337
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.5.2 (Git Hash a9302535553c73243c632ad3c4c80beec3d19a1e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_5

In [85]:
kwargs = load_image(datasets, 0)

feature_maps = model.extract_feats([kwargs['img']])[0]
cls_logits, rbox_preds = model.bbox_head.forward_multi_lvl(feature_maps)
fm_shapes = [(fm.size(2), fm.size(3)) for fm in feature_maps]
positions, strides, regress_ranges = model.bbox_head.compute_auxiliaries(fm_shapes)
cls_targets, cls_scores, reg_targets, reg_weights = model.bbox_head.compute_targets(
    kwargs['gt_bboxes'], kwargs['gt_labels'], rbox_preds, positions, strides, regress_ranges)
# format classification components
cls_logits = cls_logits.flatten(0, 1)  # B, P, C -> BP, C
cls_targets = cls_targets.flatten()  # B, P -> BP
cls_scores = cls_scores.flatten()  # B, P -> BP
# format regression components
rbox_preds = rbox_preds.flatten(0, 1)  # B, P, 5 -> BP, 5
reg_targets = reg_targets.flatten(0, 1)  # B, P, 5 -> BP, 5
reg_weights = reg_weights.flatten()  # B, P -> BP
# filter on prediction being a detection
detection_mask = cls_targets != cls_targets.max()
rbox_preds = rbox_preds[detection_mask]
reg_targets = reg_targets[detection_mask]
reg_weights = reg_weights[detection_mask]

In [87]:
print(rbox_preds[:3])
print(reg_targets[:3])
print(positions[detection_mask][:3])

tensor([[-1.3416,  2.2137, 11.2083,  7.7931, -0.0305],
        [-1.7782, -1.2409, 10.3124,  7.6003, -0.0376],
        [ 0.8997, -1.1670, 11.0492,  7.7701, -0.0703]],
       grad_fn=<SliceBackward0>)
tensor([[6.1304e+02, 3.3284e+02, 9.4340e+00, 1.4946e+01, 5.5860e-01],
        [6.2932e+02, 2.9547e+02, 8.5437e+00, 1.5005e+01, 3.6717e-01],
        [6.3866e+02, 2.9964e+02, 1.0636e+01, 1.7989e+01, 4.0489e-01]])
tensor([[612., 332.],
        [628., 292.],
        [636., 300.]])
